In [1]:
from IPython.core.display import display, HTML
display(HTML('<h1>Test Set</h1>'))



In [2]:
display(HTML('<h2>Imports</h2>'))

In [3]:
import pandas as pd
import numpy as np
from IPython.display import Image, HTML
import glob
import glob
import pandas as pd
import numpy as np
import cv2
from skimage import feature
import pickle

In [4]:
display(HTML('<h2>Load Data</h2>'))

In [5]:
full_data = pd.read_csv('x_y2.csv')

In [6]:
test_data = pd.DataFrame()
test_data['name'] = [file.split('/')[-1].split('.')[0] for file in glob.glob('TestSetImages/*.jpg')]
len(test_data)

104

In [7]:
full_test_data = pd.merge(full_data, test_data, how='inner', on=['name'])[['name','age','sex','s0','s1','s2','s3','s4','s5','s6','y0','y1','y2','y3','y4','y5','diagnosis']]

In [8]:
#full_test_data = pd.concat([full_test_data,full_test_data])
#full_test_data = pd.concat([full_test_data,full_test_data])
#full_test_data = pd.concat([full_test_data,full_test_data])
#full_test_data = pd.concat([full_test_data,full_test_data,full_test_data,full_test_data,full_test_data])
len(full_test_data)

5944

In [9]:
full_test_data.drop_duplicates(subset='name',inplace=True)

In [10]:
len(full_test_data)

104

In [11]:
full_test_data['image'] = full_test_data['name'].apply(lambda x: '<img src="./TestSetImages/'+x+'.jpg" style="width:40px">')

In [12]:
full_test_data['image'].iloc[0]

'<img src="./TestSetImages/ISIC_0000177.jpg" style="width:40px">'

In [13]:
full_test_data =pd.concat([full_test_data[:100],full_test_data[:100]])
len(full_test_data)

200

In [14]:
display(HTML('<h2>Problem Statement</h2>'))

In [15]:
HTML(full_test_data[['name','age','sex','s0','s1','s2','s3','s4','s5','diagnosis','image']].to_html(escape=False))

,name,age,sex,s0,s1,s2,s3,s4,s5,diagnosis,image
0,ISIC_0000177,1,0.0,25,17,31,31,2,19,0,"<img src=""./TestSetImages/ISIC_0000177.jpg"" st..."
1,ISIC_0000178,3,1.0,1,33,23,9,13,26,0,"<img src=""./TestSetImages/ISIC_0000178.jpg"" st..."
2,ISIC_0000179,1,0.0,27,29,10,28,7,26,0,"<img src=""./TestSetImages/ISIC_0000179.jpg"" st..."
3,ISIC_0000180,2,1.0,25,35,8,18,24,28,0,"<img src=""./TestSetImages/ISIC_0000180.jpg"" st..."
4,ISIC_0000181,0,1.0,28,36,17,3,15,13,0,"<img src=""./TestSetImages/ISIC_0000181.jpg"" st..."
5,ISIC_0000182,2,0.0,37,16,3,15,2,4,0,"<img src=""./TestSetImages/ISIC_0000182.jpg"" st..."
6,ISIC_0000183,1,1.0,36,29,20,23,30,28,0,"<img src=""./TestSetImages/ISIC_0000183.jpg"" st..."
7,ISIC_0000184,1,1.0,14,14,18,33,10,1,0,"<img src=""./TestSetImages/ISIC_0000184.jpg"" st..."
8,ISIC_0000185,1,1.0,7,14,34,32,18,12,0,"<img src=""./TestSetImages/ISIC_0000185.jpg"" st..."
9,ISIC_0000186,0,0.0,18,6,12,16,22,22,0,"<img src=""./TestSetImages/ISIC_0000186.jpg"" st..."


In [16]:
X_test = full_test_data[['name','age','sex','s0','s1','s2','s3','s4','s5','s6']]

In [17]:
y_test = full_test_data[['y0','y1','y2','y3','y4','y5']]

In [18]:
batch_size = 100
x_pixels = int(100)
y_pixels = int(100)
n_classes = 6

In [19]:
def get_data(data_x,data_y,i):
    i = i*batch_size
    x_data_cnn = []
    x_data_neural = []
       
    type_class_check = type(np.array([1,2,3]))
    for counter in range(i,i+batch_size):
        try:
            #Insert Path to file where all images are present
            image = cv2.imread('/home/pratik/Desktop/be_project/images/Data/Images/working/'+data_x.iloc[counter]['name']+'.jpg',cv2.IMREAD_GRAYSCALE)
            if type(image) == type_class_check:
                x_data_cnn.append(cv2.resize(image,(x_pixels,y_pixels)))
                
            else:
                #print('nai hai')
                x_data_cnn.append(np.zeros([x_pixels,y_pixels]))
            x_data_neural.append(data_x.iloc[counter][['age','sex','s0','s1','s2','s3','s4','s5','s6']])
        except ValueError:
            print("Missed data point")
            
    y_data = data_y[['y0','y1','y2','y3','y4','y5']][i:i+batch_size]
    #x_data = np.zeros([batch_size,x_pixels,y_pixels])
    return x_data_cnn,x_data_neural, y_data


In [20]:
display(HTML('<h2>Model</h2>'))

In [21]:
import tensorflow as tf

n_classes = 6
batch_size = 200

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

#The place holder to hold the values of new x and y at different times.
x_neural = tf.placeholder('float', [batch_size, 9])
x_cnn = tf.placeholder('float', [batch_size, x_pixels*y_pixels])
y = tf.placeholder('float',[batch_size,n_classes])

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([9, n_nodes_hl1], name='weights1')),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]),name='biases1')}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]), name='weights2'),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]), name='biases2')}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]),name='weights3'),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]), name='biases3')}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]),name='output_nn'),
                    'biases':tf.Variable(tf.random_normal([n_classes]),name='biases_output')}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    #output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']
    return l3


def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32]),name='W_conv1'),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64]),name='W_conv2'),
               'W_fc':tf.Variable(tf.random_normal([(int(x_pixels//4))*(int(y_pixels//4))*64,1024]),name='W_fc'),
               'out':tf.Variable(tf.random_normal([1024, n_classes]),name='outc')}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32]), name='b_conv1'),
               'b_conv2':tf.Variable(tf.random_normal([64]), name='b_conv2'),
               'b_fc':tf.Variable(tf.random_normal([1024]), name='b_fc'),
               'out':tf.Variable(tf.random_normal([n_classes]), name='outb')}

    x = tf.reshape(x, shape=[-1, x_pixels, y_pixels, 1])

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)

    fc = tf.reshape(conv2,[-1, (int(x_pixels//4))*(int(y_pixels//4))*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)
    #print(fc.shape)
    #output = tf.matmul(fc, weights['out'])+biases['out']
    #print(output.shape)
    return fc






def train_neural_network(x_neural,x_cnn):
    output_combined = {'weights':tf.Variable(tf.random_normal([1524, n_classes]),name='finw'),
                    'biases':tf.Variable(tf.random_normal([n_classes]),name='finb')}
    output_cnn = convolutional_neural_network(x_cnn)
    output_neural = neural_network_model(x_neural)
    #print(output_cnn.shape,output_neural.shape)
    output = tf.concat([output_cnn,output_neural],1)
    #print(output.shape)
    prediction = tf.add(tf.matmul(output,output_combined['weights']), output_combined['biases'])
    #prediction = tf.nn.relu(raw_pred)
    #prediction = neural_network_model(x_neural)
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = y) )
    #optimizer = tf.train.AdamOptimizer().minimize(cost)
    trial = tf.argmax(prediction,1)
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
    
    hm_epochs = 1
    
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        
        sess.run(tf.initialize_all_variables())
        

        saver.restore(sess, "/home/pratik/combinationfin2.ckpt")
        test_x_cnn, test_x_neural, test_y = get_data(X_test,y_test,0)
        test_x_cnn = np.array(test_x_cnn).reshape(batch_size,x_pixels*y_pixels)
        trial,accuracy = sess.run([trial,accuracy],feed_dict={x_cnn: test_x_cnn, x_neural: test_x_neural, y: test_y})
        print("\n\n Accuracy:\t", accuracy*100, "%")
        return trial
    
prediction = train_neural_network(x_neural,x_cnn)


/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use `tf.global_variables_initializer` instead.
INFO:tensorflow:Restoring parameters from /home/pratik/combinationfin2.ckpt


 Accuracy:	 86.50000095367432 %


In [22]:
display(HTML('<h2>Prediction</h2>'))

In [23]:
full_test_data['prediction'] = prediction

In [24]:
HTML(full_test_data[['name','age','sex','s0','s1','s2','s3','s4','s5','s6','diagnosis','image','prediction']].to_html(escape=False))

,name,age,sex,s0,s1,s2,s3,s4,s5,s6,diagnosis,image,prediction
0,ISIC_0000177,1,0.0,25,17,31,31,2,19,0,0,"<img src=""./TestSetImages/ISIC_0000177.jpg"" st...",0
1,ISIC_0000178,3,1.0,1,33,23,9,13,26,0,0,"<img src=""./TestSetImages/ISIC_0000178.jpg"" st...",0
2,ISIC_0000179,1,0.0,27,29,10,28,7,26,0,0,"<img src=""./TestSetImages/ISIC_0000179.jpg"" st...",0
3,ISIC_0000180,2,1.0,25,35,8,18,24,28,0,0,"<img src=""./TestSetImages/ISIC_0000180.jpg"" st...",0
4,ISIC_0000181,0,1.0,28,36,17,3,15,13,0,0,"<img src=""./TestSetImages/ISIC_0000181.jpg"" st...",0
5,ISIC_0000182,2,0.0,37,16,3,15,2,4,0,0,"<img src=""./TestSetImages/ISIC_0000182.jpg"" st...",0
6,ISIC_0000183,1,1.0,36,29,20,23,30,28,0,0,"<img src=""./TestSetImages/ISIC_0000183.jpg"" st...",0
7,ISIC_0000184,1,1.0,14,14,18,33,10,1,0,0,"<img src=""./TestSetImages/ISIC_0000184.jpg"" st...",0
8,ISIC_0000185,1,1.0,7,14,34,32,18,12,0,0,"<img src=""./TestSetImages/ISIC_0000185.jpg"" st...",0
9,ISIC_0000186,0,0.0,18,6,12,16,22,22,0,0,"<img src=""./TestSetImages/ISIC_0000186.jpg"" st...",0


In [25]:
0 --- > Nevus
1 ---> Melanoma
2 ----> Seborric keratosis
3 ----> Dermatites
4 ----> Herpes
5 ---> Kaposi Sarcoma

SyntaxError: invalid syntax (<ipython-input-25-63959dc78e95>, line 1)